# 1. Import

## 1.1. Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.colors import ListedColormap

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime as dt

from scipy.stats.mstats import normaltest

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Define palette
pal = ["#274D60","#6BA3BE","#0C969C","#0A7075","#032F30", "#031716"]
cmap = ListedColormap(pal)
palette = pal

from matplotlib.colors import LinearSegmentedColormap
pal_1 = ["#032F30","#0A7075","#0C969C","#fff9e6","#ffebc6","#fcd997","#f5b971"]
palette_1 = pal_1

reversed_pal_1 = pal_1[::-1]

cmap_1 = LinearSegmentedColormap.from_list("reversed_blue_green_cmap", reversed_pal_1)

In [3]:
# Missing values and missing values percentage
def missing_value_percentage(df):
    totalnull = df.isnull().sum().sort_values(ascending = False)[df.isnull().sum().sort_values(ascending = False) != 0]
    percentage = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100, 2)[round(df.isnull().sum().sort_values(ascending = False)/len(df)*100, 2) != 0]
    return pd.concat([totalnull, percentage], axis = 1, keys = ['TotalNull', 'Percentage'])

## 1.2. Import Data

In [5]:
trans_df=pd.read_excel('/Users/hominhtrung/Documents/Giáo Trình - FTU2/Năm 3/Phân tích dữ liệu/[PTDL] Final Project/Raw_data.xlsx', sheet_name='Transactions')
newcus_df=pd.read_excel('/Users/hominhtrung/Documents/Giáo Trình - FTU2/Năm 3/Phân tích dữ liệu/[PTDL] Final Project/Raw_data.xlsx', sheet_name='NewCustomerList')
cusdemo_df=pd.read_excel('/Users/hominhtrung/Documents/Giáo Trình - FTU2/Năm 3/Phân tích dữ liệu/[PTDL] Final Project/Raw_data.xlsx', sheet_name='CustomerDemographic')
cusad_df=pd.read_excel('/Users/hominhtrung/Documents/Giáo Trình - FTU2/Năm 3/Phân tích dữ liệu/[PTDL] Final Project/Raw_data.xlsx', sheet_name='CustomerAddress')

In [6]:
trans_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [7]:
newcus_df.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.74,0.925,1.15625,0.982812,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.46,0.460,0.57500,0.488750,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28 00:00:00,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,1.06,1.060,1.06000,1.060000,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.92,1.150,1.15000,1.150000,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,1.07,1.070,1.33750,1.337500,4,4,1.703125


In [8]:
cusdemo_df.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


In [9]:
cusad_df.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


# **2. Data Cleaning**

## 2.1. *Transactions*

In [10]:
trans_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [11]:
trans_df.shape

(20000, 13)

> Checking data type

In [12]:
trans_df.dtypes

transaction_id                      int64
product_id                          int64
customer_id                         int64
transaction_date           datetime64[ns]
online_order                      float64
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date           float64
dtype: object

In [13]:
for col in trans_df.columns:
    if trans_df[col].nunique() < 10:
        print(f"{col}: {trans_df[col].unique()}")

online_order: [ 0.  1. nan]
order_status: ['Approved' 'Cancelled']
brand: ['Solex' 'Trek Bicycles' 'OHM Cycles' 'Norco Bicycles' 'Giant Bicycles'
 'WeareA2B' nan]
product_line: ['Standard' 'Road' 'Mountain' 'Touring' nan]
product_class: ['medium' 'low' 'high' nan]
product_size: ['medium' 'large' 'small' nan]


In [14]:
trans_df['transaction_id']=trans_df['transaction_id'].astype('object')
trans_df['product_id']=trans_df['product_id'].astype('object')
trans_df['customer_id']=trans_df['customer_id'].astype('object')
trans_df['online_order']=trans_df['online_order'].map({1: True, 0: False})

In [15]:
trans_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [16]:
trans_df['transaction_date'] = pd.to_datetime(trans_df['transaction_date'])

In [17]:
trans_df['transaction_date'].sort_values()

516     2017-01-01
5876    2017-01-01
3459    2017-01-01
12484   2017-01-01
19130   2017-01-01
           ...    
605     2017-12-30
15269   2017-12-30
19906   2017-12-30
15756   2017-12-30
12003   2017-12-30
Name: transaction_date, Length: 20000, dtype: datetime64[ns]

In [18]:
# today = pd.Timestamp('2018-01-01')
# today

In [19]:
trans_df['product_first_sold_date'] = pd.to_timedelta(trans_df['product_first_sold_date'], unit='D') + pd.Timestamp('1900-01-01 00:00:00')

In [20]:
trans_df[trans_df['product_first_sold_date'].notna()]['product_first_sold_date'].sort_values()

13882   1991-01-23
13411   1991-01-23
1548    1991-01-23
11494   1991-01-23
10036   1991-01-23
           ...    
10145   2016-12-08
1051    2016-12-08
2066    2016-12-08
15252   2016-12-08
5505    2016-12-08
Name: product_first_sold_date, Length: 19803, dtype: datetime64[ns]

In [21]:
trans_df

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-04
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-05
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-22
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-18
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,2003-07-23
19996,19997,41,127,2017-11-09,True,Approved,Solex,Road,medium,medium,416.98,312.74,1997-05-12
19997,19998,87,2284,2017-04-14,True,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,2010-08-22
19998,19999,6,2764,2017-07-03,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,2004-08-19


> Checking duplicate

In [22]:
#Checking duplicate
duplicate_rows = trans_df[trans_df.duplicated()]

print("Duplicate Rows:")
print(duplicate_rows)

num_duplicates = len(duplicate_rows)
print(f"\nNumber of duplicate rows: {num_duplicates}")

Duplicate Rows:
Empty DataFrame
Columns: [transaction_id, product_id, customer_id, transaction_date, online_order, order_status, brand, product_line, product_class, product_size, list_price, standard_cost, product_first_sold_date]
Index: []

Number of duplicate rows: 0


> Checking missing value

In [23]:
missing_value_percentage(trans_df)

,TotalNull,Percentage
online_order,360,1.80
brand,197,0.98
product_line,197,0.98
product_class,197,0.98
product_size,197,0.98
standard_cost,197,0.98
product_first_sold_date,197,0.98


In [24]:
trans_df[trans_df['brand'].isnull()]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
136,137,0,431,2017-09-23,False,Approved,NaN,NaN,NaN,NaN,1942.61,NaN,NaT
159,160,0,3300,2017-08-27,False,Approved,NaN,NaN,NaN,NaN,1656.86,NaN,NaT
366,367,0,1614,2017-03-10,False,Approved,NaN,NaN,NaN,NaN,850.89,NaN,NaT
406,407,0,2559,2017-06-14,True,Approved,NaN,NaN,NaN,NaN,710.59,NaN,NaT
676,677,0,2609,2017-07-02,False,Approved,NaN,NaN,NaN,NaN,1972.01,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19340,19341,0,443,2017-12-26,True,Approved,NaN,NaN,NaN,NaN,744.54,NaN,NaT
19383,19384,0,2407,2017-06-11,False,Approved,NaN,NaN,NaN,NaN,1098.18,NaN,NaT
19793,19794,0,2860,2017-01-13,False,Approved,NaN,NaN,NaN,NaN,868.56,NaN,NaT
19859,19860,0,2468,2017-06-24,True,Approved,NaN,NaN,NaN,NaN,1497.43,NaN,NaT


It can be seen that the columns `brand`, `product_line`, `product_class`, `product_size`, `standard_cost`, `product_first_sold_date` are null in different unique rows, so the authors decided to remove these rows without using any filling method.

In [25]:
trans_df[trans_df['brand'].isnull()][['brand', 'product_line', 'product_class', 'product_size', 'standard_cost', 'product_first_sold_date']].drop_duplicates()

,brand,product_line,product_class,product_size,standard_cost,product_first_sold_date
136,NaN,NaN,NaN,NaN,NaN,NaT


In [26]:
trans_df.drop(index=trans_df[trans_df['brand'].isnull()][['brand', 'product_line', 'product_class', 'product_size', 'standard_cost', 'product_first_sold_date']].index, axis=0, inplace=True)

In [27]:
trans_df.shape

(19803, 13)

In [28]:
# Filling online_order missing value
trans_df['online_order'] = trans_df['online_order'].fillna(trans_df['online_order'].mode()[0])

In [29]:
missing_value_percentage(trans_df)

,TotalNull,Percentage


In [30]:
trans_df.dtypes

transaction_id                     object
product_id                         object
customer_id                        object
transaction_date           datetime64[ns]
online_order                         bool
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date    datetime64[ns]
dtype: object

In [31]:
trans_df.select_dtypes(include='object').columns

Index(['transaction_id', 'product_id', 'customer_id', 'order_status', 'brand',
       'product_line', 'product_class', 'product_size'],
      dtype='object')

In [32]:
trans_category_cols= ['order_status', 'brand','product_line', 'product_class', 'product_size']

In [33]:
for col in trans_category_cols:
    print(trans_df[col].value_counts())

order_status
Approved     19625
Cancelled      178
Name: count, dtype: int64
brand
Solex             4253
Giant Bicycles    3312
WeareA2B          3295
OHM Cycles        3043
Trek Bicycles     2990
Norco Bicycles    2910
Name: count, dtype: int64
product_line
Standard    14176
Road         3970
Touring      1234
Mountain      423
Name: count, dtype: int64
product_class
medium    13826
high       3013
low        2964
Name: count, dtype: int64
product_size
medium    12990
large      3976
small      2837
Name: count, dtype: int64


In [34]:
transactions=trans_df.copy()

In [35]:
transactions

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-04
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-05
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-22
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-18
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,2003-07-23
19996,19997,41,127,2017-11-09,True,Approved,Solex,Road,medium,medium,416.98,312.74,1997-05-12
19997,19998,87,2284,2017-04-14,True,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,2010-08-22
19998,19999,6,2764,2017-07-03,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,2004-08-19


transactions.dtypes

In [41]:
transactions.describe(include=['int64', 'float64']).T

,count,mean,std,min,25%,50%,75%,max
list_price,19803.0,1107.997866,582.77046,12.01,575.27,1163.89,1635.3,2091.47
standard_cost,19803.0,556.046951,405.95566,7.21,215.14,507.58,795.1,1759.85


In [112]:
transactions.describe().T

,count,mean,min,25%,50%,75%,max,std
transaction_date,19803,2017-07-01 15:40:43.811543552,2017-01-01 00:00:00,2017-04-01 00:00:00,2017-07-03 00:00:00,2017-10-02 00:00:00,2017-12-30 00:00:00,NaN
list_price,19803.0,1107.997866,12.01,575.27,1163.89,1635.3,2091.47,582.77046
standard_cost,19803.0,556.046951,7.21,215.14,507.58,795.1,1759.85,405.95566
product_first_sold_date,19803,2004-08-02 18:38:13.834267520,1991-01-23 00:00:00,1997-08-27 00:00:00,2004-08-19 00:00:00,2011-05-11 00:00:00,2016-12-08 00:00:00,NaN


In [43]:
transactions.describe(include=['object']).T

,count,unique,top,freq
transaction_id,19803,19803,1,1
product_id,19803,101,0,1181
customer_id,19803,3494,1068,14
order_status,19803,2,Approved,19625
brand,19803,6,Solex,4253
product_line,19803,4,Standard,14176
product_class,19803,3,medium,13826
product_size,19803,3,medium,12990


In [56]:
transactions.to_csv('/Users/hominhtrung/Documents/Giáo Trình - FTU2/Năm 3/Phân tích dữ liệu/[PTDL] Final Project/transaction.csv')

## *2.2. New Customers*

In [44]:
newcus_df.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.74,0.925,1.15625,0.982812,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.46,0.460,0.57500,0.488750,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28 00:00:00,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,1.06,1.060,1.06000,1.060000,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.92,1.150,1.15000,1.150000,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,1.07,1.070,1.33750,1.337500,4,4,1.703125


In [45]:
newcus=newcus_df.copy()

In [46]:
# Drop Irrelevant columns
newcus.drop(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'], axis=1, inplace=True)

In [47]:
newcus

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28 00:00:00,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,9,2 Sloan Way,2200,NSW,Australia,7,996,0.374000
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,6,04 Union Crossing,2196,NSW,Australia,10,997,0.357000
997,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,15,33475 Fair Oaks Junction,4702,QLD,Australia,2,997,0.357000
998,Dickie,Cubbini,Male,30,1952-12-17,Financial Advisor,Financial Services,Mass Customer,N,Yes,19,57666 Victoria Way,4215,QLD,Australia,2,997,0.357000


> Checking data type

In [48]:
newcus.dtypes

first_name                              object
last_name                               object
gender                                  object
past_3_years_bike_related_purchases      int64
DOB                                     object
job_title                               object
job_industry_category                   object
wealth_segment                          object
deceased_indicator                      object
owns_car                                object
tenure                                   int64
address                                 object
postcode                                 int64
state                                   object
country                                 object
property_valuation                       int64
Rank                                     int64
Value                                  float64
dtype: object

In [49]:
for col in newcus.columns:
    if newcus[col].nunique() < 10:
        print(f"{col}: {newcus[col].unique()}")

gender: ['Male' 'Female' 'U']
job_industry_category: ['Manufacturing' 'Property' 'Financial Services' 'Entertainment' 'Retail'
 'IT' 'Telecommunications' 'Health' nan 'Argiculture']
wealth_segment: ['Mass Customer' 'Affluent Customer' 'High Net Worth']
deceased_indicator: ['N']
owns_car: ['Yes' 'No']
state: ['QLD' 'NSW' 'VIC']
country: ['Australia']


In [50]:
newcus['gender'].value_counts()

gender
Female    513
Male      470
U          17
Name: count, dtype: int64

In [51]:
newcus['DOB'] = pd.to_datetime(newcus['DOB'])

> Checking duplicate

In [52]:
duplicate_rows = newcus[newcus.duplicated()]

print("Duplicate Rows:")
print(duplicate_rows)

num_duplicates = len(duplicate_rows)
print(f"\nNumber of duplicate rows: {num_duplicates}")

Duplicate Rows:
Empty DataFrame
Columns: [first_name, last_name, gender, past_3_years_bike_related_purchases, DOB, job_title, job_industry_category, wealth_segment, deceased_indicator, owns_car, tenure, address, postcode, state, country, property_valuation, Rank, Value]
Index: []

Number of duplicate rows: 0


> Checking missing value

In [53]:
missing_value_percentage(newcus)

,TotalNull,Percentage
job_industry_category,165,16.5
job_title,106,10.6
last_name,29,2.9
DOB,17,1.7


In [54]:
# Filling missing value
newcus['job_industry_category'].fillna('Others', inplace=True, axis=0)
newcus['job_title'].fillna('Others', inplace=True, axis=0)
newcus['last_name'].fillna('Unknown', inplace=True)

In [55]:
newcus[newcus['DOB'].isnull()]

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
59,Normy,Goodinge,U,5,NaT,Associate Professor,IT,Mass Customer,N,No,4,7232 Fulton Parkway,3810,VIC,Australia,5,57,1.375000
226,Hatti,Carletti,U,35,NaT,Legal Assistant,IT,Affluent Customer,N,Yes,11,6 Iowa Center,2519,NSW,Australia,9,226,1.112500
324,Rozamond,Turtle,U,69,NaT,Legal Assistant,IT,Mass Customer,N,Yes,3,57025 New Castle Street,3850,VIC,Australia,3,324,1.010000
358,Tamas,Swatman,U,65,NaT,Assistant Media Planner,Entertainment,Affluent Customer,N,No,5,78 Clarendon Drive,4551,QLD,Australia,8,358,0.980000
360,Tracy,Andrejevic,U,71,NaT,Programmer II,IT,Mass Customer,N,Yes,11,5675 Burning Wood Trail,3030,VIC,Australia,7,361,0.977500
374,Agneta,McAmish,U,66,NaT,Structural Analysis Engineer,IT,Mass Customer,N,No,15,5773 Acker Way,4207,QLD,Australia,6,375,0.960000
434,Gregg,Aimeric,U,52,NaT,Internal Auditor,IT,Mass Customer,N,No,7,72423 Surrey Street,3753,VIC,Australia,5,433,0.906250
439,Johna,Bunker,U,93,NaT,Tax Accountant,IT,Mass Customer,N,Yes,14,3686 Waubesa Way,3065,VIC,Australia,6,436,0.903125
574,Harlene,Nono,U,69,NaT,Human Resources Manager,IT,Mass Customer,N,No,12,0307 Namekagon Crossing,2170,NSW,Australia,7,575,0.796875
598,Gerianne,Kaysor,U,15,NaT,Project Manager,IT,Affluent Customer,N,No,5,882 Toban Lane,2121,NSW,Australia,11,599,0.775000


In [56]:
DOB_index_drop = newcus[newcus['DOB'].isnull()].index
newcus.drop(index=DOB_index_drop, inplace=True, axis=0)

In [57]:
newcus

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,9,2 Sloan Way,2200,NSW,Australia,7,996,0.374000
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,6,04 Union Crossing,2196,NSW,Australia,10,997,0.357000
997,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,15,33475 Fair Oaks Junction,4702,QLD,Australia,2,997,0.357000
998,Dickie,Cubbini,Male,30,1952-12-17,Financial Advisor,Financial Services,Mass Customer,N,Yes,19,57666 Victoria Way,4215,QLD,Australia,2,997,0.357000


In [58]:
newcus.select_dtypes(include='object').columns

Index(['first_name', 'last_name', 'gender', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'address', 'state', 'country'],
      dtype='object')

In [59]:
newcus_category_cols=['gender','job_industry_category','wealth_segment','deceased_indicator','owns_car','state','country']

In [60]:
for col in newcus_category_cols:
    print(newcus[col].value_counts())

gender
Female    513
Male      470
Name: count, dtype: int64
job_industry_category
Financial Services    202
Manufacturing         199
Others                165
Health                152
Retail                 78
Property               64
Entertainment          36
IT                     36
Argiculture            26
Telecommunications     25
Name: count, dtype: int64
wealth_segment
Mass Customer        499
High Net Worth       249
Affluent Customer    235
Name: count, dtype: int64
deceased_indicator
N    983
Name: count, dtype: int64
owns_car
No     497
Yes    486
Name: count, dtype: int64
state
NSW    499
VIC    258
QLD    226
Name: count, dtype: int64
country
Australia    983
Name: count, dtype: int64


In [ ]:
newcus['deceased_indicator'] = newcus['deceased_indicator'].replace('N', 'No')

In [62]:
newcus.dtypes

first_name                                     object
last_name                                      object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
owns_car                                       object
tenure                                          int64
address                                        object
postcode                                        int64
state                                          object
country                                        object
property_valuation                              int64
Rank                                            int64
Value                                         float64
dtype: object

In [63]:
newcus

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,No,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,No,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,No,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,No,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,No,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,No,No,9,2 Sloan Way,2200,NSW,Australia,7,996,0.374000
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,No,No,6,04 Union Crossing,2196,NSW,Australia,10,997,0.357000
997,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,No,Yes,15,33475 Fair Oaks Junction,4702,QLD,Australia,2,997,0.357000
998,Dickie,Cubbini,Male,30,1952-12-17,Financial Advisor,Financial Services,Mass Customer,No,Yes,19,57666 Victoria Way,4215,QLD,Australia,2,997,0.357000


In [64]:
newcus['state'].value_counts()

state
NSW    499
VIC    258
QLD    226
Name: count, dtype: int64

In [65]:
newcustomerlist=newcus.copy()

In [67]:
newcustomerlist.describe(include=['int64', 'float64']).T

,count,mean,std,min,25%,50%,75%,max
past_3_years_bike_related_purchases,983.0,49.824008,27.786108,0.00,27.000000,51.00,72.000000,99.00000
tenure,983.0,11.459817,5.006123,1.00,8.000000,11.00,15.000000,22.00000
postcode,983.0,3017.740590,849.381760,2000.00,2209.000000,2795.00,3850.500000,4879.00000
property_valuation,983.0,7.406918,2.758293,1.00,6.000000,8.00,9.000000,12.00000
Rank,983.0,497.745677,289.186799,1.00,248.500000,500.00,748.000000,1000.00000
Value,983.0,0.882926,0.294166,0.34,0.650781,0.86,1.076953,1.71875


In [68]:
newcustomerlist.describe(include=['object']).T

,count,unique,top,freq
first_name,983,925,Dorian,3
last_name,983,945,Unknown,29
gender,983,2,Female,513
job_title,983,185,Others,105
job_industry_category,983,10,Financial Services,202
wealth_segment,983,3,Mass Customer,499
deceased_indicator,983,1,No,983
owns_car,983,2,No,497
address,983,983,45 Shopko Center,1
state,983,3,NSW,499


In [66]:
newcustomerlist.to_csv('/Users/hominhtrung/Documents/Giáo Trình - FTU2/Năm 3/Phân tích dữ liệu/[PTDL] Final Project/newcustomerlist.csv')

## *2.3. Customer Address*

In [70]:
cusad=cusad_df.copy()
cusad

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9
...,...,...,...,...,...,...
3994,3999,1482 Hauk Trail,3064,VIC,Australia,3
3995,4000,57042 Village Green Point,4511,QLD,Australia,6
3996,4001,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,4002,8194 Lien Street,4032,QLD,Australia,7


In [71]:
cusad.shape

(3999, 6)

> Checking Data Type

In [72]:
cusad.dtypes

customer_id            int64
address               object
postcode               int64
state                 object
country               object
property_valuation     int64
dtype: object

In [73]:
cusad['customer_id']=cusad['customer_id'].astype('object')

> Checking duplicate

In [74]:
duplicate_rows = cusad[cusad.duplicated()]

print("Duplicate Rows:")
print(duplicate_rows)

num_duplicates = len(duplicate_rows)
print(f"\nNumber of duplicate rows: {num_duplicates}")

Duplicate Rows:
Empty DataFrame
Columns: [customer_id, address, postcode, state, country, property_valuation]
Index: []

Number of duplicate rows: 0


> Checking missing value

In [75]:
missing_value_percentage(cusad)

,TotalNull,Percentage


In [76]:
cusad.select_dtypes(include='object').columns

Index(['customer_id', 'address', 'state', 'country'], dtype='object')

In [77]:
cusad_category_columns=['state', 'country']

In [78]:
for col in cusad_category_columns:
    print(cusad[col].value_counts())

state
NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: count, dtype: int64
country
Australia    3999
Name: count, dtype: int64


In [79]:
replace_dict = {
    'New South Wales' : 'NSW',
    'Victoria' : 'VIC',
    'Queensland' : 'QLD'
}

cusad['state'] = cusad['state'].replace(replace_dict)

In [80]:
for col in cusad_category_columns:
    print(cusad[col].value_counts())

state
NSW    2140
VIC    1021
QLD     838
Name: count, dtype: int64
country
Australia    3999
Name: count, dtype: int64


In [86]:
cusad.describe(include=['int64','float64']).T

,count,mean,std,min,25%,50%,75%,max
postcode,3999.0,2985.755939,844.878364,2000.0,2200.0,2768.0,3750.0,4883.0
property_valuation,3999.0,7.514379,2.824663,1.0,6.0,8.0,10.0,12.0


In [85]:
cusad.describe(include=['object']).T

,count,unique,top,freq
customer_id,3999,3999,1,1
address,3999,3996,3 Mariners Cove Terrace,2
state,3999,3,NSW,2140
country,3999,1,Australia,3999


In [81]:
cusad.to_csv('/Users/hominhtrung/Documents/Giáo Trình - FTU2/Năm 3/Phân tích dữ liệu/[PTDL] Final Project/customeraddress.csv')

## *2.4. Customer Demographic*

In [87]:
cusdemo=cusdemo_df.copy()
cusdemo

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09 00:00:00,VP Product Management,Health,Mass Customer,N,-100,No,19.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13 00:00:00,Statistician II,Manufacturing,High Net Worth,N,â¦testâ§,Yes,1.0
3997,3998,Sarene,Woolley,U,60,NaN,Assistant Manager,IT,High Net Worth,N,NaN,No,NaN
3998,3999,Patrizius,NaN,Male,11,1973-10-24 00:00:00,NaN,Manufacturing,Affluent Customer,N,Â¡â¢Â£Â¢âÂ§Â¶â¢ÂªÂºââ,Yes,10.0


In [88]:
cusdemo.shape

(4000, 13)

> Checking data type

In [89]:
cusdemo.dtypes

customer_id                              int64
first_name                              object
last_name                               object
gender                                  object
past_3_years_bike_related_purchases      int64
DOB                                     object
job_title                               object
job_industry_category                   object
wealth_segment                          object
deceased_indicator                      object
default                                 object
owns_car                                object
tenure                                 float64
dtype: object

In [90]:
cusdemo['tenure'].isnull().sum()

87

In [91]:
cusdemo['customer_id']=cusdemo['customer_id'].astype('object')
cusdemo['DOB'] = pd.to_datetime(cusdemo['DOB'])
# cusdemo['tenure'] = cusdemo['tenure'].astype('int64')

In [92]:
cusdemo.dtypes

customer_id                                    object
first_name                                     object
last_name                                      object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
default                                        object
owns_car                                       object
tenure                                        float64
dtype: object

In [93]:
cusdemo

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,-100,No,19.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,â¦testâ§,Yes,1.0
3997,3998,Sarene,Woolley,U,60,NaT,Assistant Manager,IT,High Net Worth,N,NaN,No,NaN
3998,3999,Patrizius,NaN,Male,11,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Â¡â¢Â£Â¢âÂ§Â¶â¢ÂªÂºââ,Yes,10.0


> Checking Duplicate

In [94]:
duplicate_rows = cusdemo[cusdemo.duplicated()]

print("Duplicate Rows:")
print(duplicate_rows)

num_duplicates = len(duplicate_rows)
print(f"\nNumber of duplicate rows: {num_duplicates}")

Duplicate Rows:
Empty DataFrame
Columns: [customer_id, first_name, last_name, gender, past_3_years_bike_related_purchases, DOB, job_title, job_industry_category, wealth_segment, deceased_indicator, default, owns_car, tenure]
Index: []

Number of duplicate rows: 0


> Checking missing value

In [95]:
missing_value_percentage(cusdemo)

,TotalNull,Percentage
job_industry_category,656,16.40
job_title,506,12.65
default,302,7.55
last_name,125,3.12
DOB,87,2.17
tenure,87,2.17


In [96]:
cusdemo['job_industry_category'].fillna('Others', inplace=True, axis=0)
cusdemo['job_title'].fillna('Others', inplace=True, axis=0)
cusdemo['last_name'].fillna('Unknown', inplace=True)
cusdemo.drop('default', axis=1, inplace=True)

In [97]:
cusdemo[cusdemo['DOB'].isnull()]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
143,144,Jory,Barrabeale,U,71,NaT,Environmental Tech,IT,Mass Customer,N,No,NaN
167,168,Reggie,Broggetti,U,8,NaT,General Manager,IT,Affluent Customer,N,Yes,NaN
266,267,Edgar,Buckler,U,53,NaT,Others,IT,High Net Worth,N,No,NaN
289,290,Giorgio,Kevane,U,42,NaT,Senior Sales Associate,IT,Mass Customer,N,No,NaN
450,451,Marlow,Flowerdew,U,37,NaT,Quality Control Specialist,IT,High Net Worth,N,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3778,3779,Ulick,Daspar,U,68,NaT,Others,IT,Affluent Customer,N,No,NaN
3882,3883,Nissa,Conrad,U,35,NaT,Legal Assistant,IT,Mass Customer,N,No,NaN
3930,3931,Kylie,Epine,U,19,NaT,Others,IT,High Net Worth,N,Yes,NaN
3934,3935,Teodor,Alfonsini,U,72,NaT,Others,IT,High Net Worth,N,Yes,NaN


In [98]:
cusdemo[cusdemo['gender']=='U']

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
33,34,Jephthah,Bachmann,U,59,1843-12-21,Legal Assistant,IT,Affluent Customer,N,No,20.0
143,144,Jory,Barrabeale,U,71,NaT,Environmental Tech,IT,Mass Customer,N,No,NaN
167,168,Reggie,Broggetti,U,8,NaT,General Manager,IT,Affluent Customer,N,Yes,NaN
266,267,Edgar,Buckler,U,53,NaT,Others,IT,High Net Worth,N,No,NaN
289,290,Giorgio,Kevane,U,42,NaT,Senior Sales Associate,IT,Mass Customer,N,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3778,3779,Ulick,Daspar,U,68,NaT,Others,IT,Affluent Customer,N,No,NaN
3882,3883,Nissa,Conrad,U,35,NaT,Legal Assistant,IT,Mass Customer,N,No,NaN
3930,3931,Kylie,Epine,U,19,NaT,Others,IT,High Net Worth,N,Yes,NaN
3934,3935,Teodor,Alfonsini,U,72,NaT,Others,IT,High Net Worth,N,Yes,NaN


In [99]:
DOB_index_drop = cusdemo[(cusdemo['DOB'].isnull()) | (cusdemo['DOB'] == '1843-12-21')].index
cusdemo.drop(index=DOB_index_drop, inplace=True, axis=0)

In [100]:
cusdemo

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0
3,4,Talbot,Unknown,Male,33,1961-10-03,Others,IT,Mass Customer,N,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Others,Affluent Customer,N,Yes,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3994,3995,Rusty,Iapico,Male,93,1975-12-12,Staff Scientist,Manufacturing,Mass Customer,N,Yes,14.0
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,No,19.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,1.0
3998,3999,Patrizius,Unknown,Male,11,1973-10-24,Others,Manufacturing,Affluent Customer,N,Yes,10.0


In [101]:
cusdemo.select_dtypes(include='object').columns

Index(['customer_id', 'first_name', 'last_name', 'gender', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car'],
      dtype='object')

In [102]:
cusdemo_category_columns=['gender', 'job_title','job_industry_category', 'wealth_segment', 'deceased_indicator', 'owns_car']

In [103]:
for col in cusdemo_category_columns:
    print(cusdemo[col].value_counts())

gender
Female    2037
Male      1872
F            1
Femal        1
M            1
Name: count, dtype: int64
job_title
Others                                  497
Business Systems Development Analyst     43
Tax Accountant                           43
Social Worker                            42
Recruiting Manager                       41
                                       ... 
Database Administrator II                 4
Health Coach III                          3
Health Coach I                            3
Research Assistant III                    3
Developer I                               1
Name: count, Length: 196, dtype: int64
job_industry_category
Manufacturing         796
Financial Services    767
Others                656
Health                596
Retail                358
Property              267
IT                    151
Entertainment         136
Argiculture           113
Telecommunications     72
Name: count, dtype: int64
wealth_segment
Mass Customer        1954
High Net W

In [104]:
cusdemo['deceased_indicator'] = cusdemo['deceased_indicator'].replace('N', 'No')
cusdemo['gender'] = cusdemo['gender'].replace({'M': 'Male', 'F': 'Female', 'Femal': 'Female'})

In [105]:
cusdemo

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,No,Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,No,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,No,Yes,15.0
3,4,Talbot,Unknown,Male,33,1961-10-03,Others,IT,Mass Customer,No,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Others,Affluent Customer,No,Yes,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3994,3995,Rusty,Iapico,Male,93,1975-12-12,Staff Scientist,Manufacturing,Mass Customer,No,Yes,14.0
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,No,No,19.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,No,Yes,1.0
3998,3999,Patrizius,Unknown,Male,11,1973-10-24,Others,Manufacturing,Affluent Customer,No,Yes,10.0


In [106]:
customerdemographic=cusdemo.copy()

In [107]:
customerdemographic['tenure']=customerdemographic['tenure'].astype('int64')

In [108]:
customerdemographic

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,No,Yes,11
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,No,Yes,16
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,No,Yes,15
3,4,Talbot,Unknown,Male,33,1961-10-03,Others,IT,Mass Customer,No,No,7
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Others,Affluent Customer,No,Yes,8
...,...,...,...,...,...,...,...,...,...,...,...,...
3994,3995,Rusty,Iapico,Male,93,1975-12-12,Staff Scientist,Manufacturing,Mass Customer,No,Yes,14
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,No,No,19
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,No,Yes,1
3998,3999,Patrizius,Unknown,Male,11,1973-10-24,Others,Manufacturing,Affluent Customer,No,Yes,10


In [110]:
customerdemographic.describe(include=['int64','float64']).T

,count,mean,std,min,25%,50%,75%,max
past_3_years_bike_related_purchases,3912.0,49.039366,28.768226,0.0,24.0,48.0,73.0,99.0
tenure,3912.0,10.654652,5.658898,1.0,6.0,11.0,15.0,22.0


In [111]:
customerdemographic.describe(include='object').T

,count,unique,top,freq
customer_id,3912,3912,1,1
first_name,3912,3078,Tobe,5
last_name,3912,3647,Unknown,121
gender,3912,2,Female,2039
job_title,3912,196,Others,497
job_industry_category,3912,10,Manufacturing,796
wealth_segment,3912,3,Mass Customer,1954
deceased_indicator,3912,2,No,3910
owns_car,3912,2,Yes,1974


In [109]:
customerdemographic.to_csv('customerdemographic.csv')